In [1]:
import pandas as pd
from topic_modelling import get_topic_model

df = pd.read_csv('data/bills_18_23.csv', index_col=0)

df.head()

/Users/yangxinyuxie/Desktop/anti-trans-bills/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Date,State,Bill,Bill Description,Bill Type,Status,Bill Link,year
1,2020-03-10,AL,SB 219,Referred to committee 3/10/20,Healthcare,Introduced,NaN,2020.0
2,2020-02-27,AL,HB 303,Introduced 2/27/20,Healthcare,Introduced,NaN,2020.0
3,2020-02-13,CO,HB 20-1114,Postponed Indefinitely 2/13/20,Healthcare,Dead/Failed,NaN,2020.0
4,2020-03-14,FL,SB 1864,Dead/Failed in committee 3/14/20,Healthcare,Dead/Failed,NaN,2020.0
5,2020-03-14,FL,HB 1365,Dead/Failed in committee 3/14/20,Healthcare,Dead/Failed,NaN,2020.0


In [2]:
df['Bill Type'].unique()

array(['Healthcare', 'Public Facilities', 'Youth Athletics', 'ID Updates',
       'Other', 'Religious Freedom', 'Nondiscrimination Protections',
       'Schools / Education', 'Drag Performance'], dtype=object)

In [13]:
docs, embeddings, topic_model, topic_df, info_tab = get_topic_model('Healthcare')

In [14]:
topic_model.visualize_hierarchy(custom_labels=True)

In [15]:
# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
from umap import UMAP
reduced_embeddings = UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)

In [16]:
# Visualize the documents in 2-dimensional space and show the titles on hover instead of the abstracts
# NOTE: You can hide the hover with `hide_document_hover=True` which is especially helpful if you have a large dataset
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings, custom_labels=True)